In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
import time 
import re 
import json 
import pandas as pd
from datetime import datetime

## id 와 password 입력

In [2]:
user_id=""
user_passwd="" 

In [3]:
login_option="instagram" # facebook or instagram 
driver_path="chromedriver.exe" 
instagram_id_name="username" 
instagram_pw_name="password"
instagram_login_btn=".sqdOP.L3NKy.y3zKF "
instagram_information_save_name = ".sqdOP.yWX7d.y3zKF"
instagram_alarm_name = ".aOOlW.HoLwm"
is_information_save = False
is_alarm_setting = False

driver = webdriver.Chrome("./chromedriver")

In [4]:
print(f"login start - option {login_option}")
login_url = "https://www.instagram.com/accounts/login/"
driver.get(login_url)
print(login_option)

login start - option instagram
instagram


In [5]:
if login_option == "instagram": 
    time.sleep(2)
    try: 
        instagram_id_form = driver.find_element_by_name(instagram_id_name) 
        instagram_id_form.send_keys(user_id) 
        
        instagram_pw_form = driver.find_element_by_name(instagram_pw_name) 
        instagram_pw_form.send_keys(user_passwd) 
        
        login_ok_button = driver.find_element_by_css_selector(instagram_login_btn) 
        login_ok_button.click() 
        is_login_success = True
        
        time.sleep(3)
        if is_information_save == False and is_alarm_setting == False:
            information_save_btn = driver.find_element_by_css_selector(instagram_information_save_name)
            information_save_btn.click()
            is_information_save = True
            
            alarm_setting_btn = driver.find_element_by_css_selector(instagram_alarm_name)
            alarm_setting_btn.click()
            is_alarm_setting = True
            
    except: 
        print("instagram login fail") 
        is_login_success = False

instagram login fail


## 위 내용까지 로그인

In [11]:
my_account_xpath = "/html/body/div[1]/section/nav/div[2]/div/div/div[3]/div/div[6]/span"
my_account_button = driver.find_element_by_xpath(my_account_xpath)
my_account_button.click()

my_profile_full_xpath = "/html/body/div[1]/section/nav/div[2]/div/div/div[3]/div/div[6]/div[2]/div[2]/div[2]/a[1]/div"
my_profile_button = driver.find_element_by_xpath(my_profile_full_xpath)
my_profile_button.click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/section/nav/div[2]/div/div/div[3]/div/div[6]/span"}
  (Session info: chrome=98.0.4758.102)


In [ ]:
time.sleep(1.5)
# 게시물 개수
post_count = driver.find_element_by_xpath("/html/body/div[1]/section/main/div/header/section/ul/li[1]/div/span").text
post_count = int(post_count)

print("게시물 수 : ",post_count)

In [ ]:
# 첫 게시물
first_post_css = "a > div.eLAPa > div._9AhH0"
first_post_button = driver.find_element_by_css_selector(first_post_css)
first_post_button.click()

while True:
    try:
        button = driver.find_element_by_css_selector('body > div._2dDPU.CkGkG > div.zZYga > div > article > div.eo2As > div.EtaWk > ul > li > div > button > span')
    except:
        pass

    if button is not None:
        try:
            driver.find_element_by_css_selector('body > div._2dDPU.CkGkG > div.zZYga > div > article > div.eo2As > div.EtaWk > ul > li > div > button > span').click()
        except:
            break

# 대댓글 버튼 누르기
buttons = driver.find_elements_by_css_selector('li > ul > li > div > button')

for button in buttons:
    button.send_keys(Keys.ENTER)

In [185]:
# 첫 게시물
first_post_css = "a > div.eLAPa > div._9AhH0"
first_post_button = driver.find_element_by_css_selector(first_post_css)
first_post_button.click()

for i in range(post_count-1) :
    # 날짜 데이터 크롤링
    time.sleep(1.5)
    #date = driver.find_element_by_css_selector("div.k_Q0X.I0_K8.NnvRN > a > time").text
    date = driver.find_element_by_css_selector('time.FH9sR.Nzb55')
    date_info = pd.to_datetime(date.get_attribute('datetime')).normalize()
    date = str(date_info.year) + "년 " + str(date_info.month) + "월 " + str(date_info.day) +"일"
    print("date : ", date)

    # 장소 크롤링
    time.sleep(0.5)
    try : 
        place = driver.find_element_by_css_selector("body > div.RnEpo._Yhr4 > div.pbNvD.QZZGH.bW6vo > div > article > div > div.HP0qD > div > div > div.UE9AK > div > header > div.o-MQd.z8cbW > div.M30cS > div > div.JF9hh > div > a").text
    except : 
        place = None
    print("place : ",place)
    
    # 게시글 내용 크롤링
    #body > div.RnEpo._Yhr4 > div.pbNvD.QZZGH.bW6vo > div > article > div > div.HP0qD > div > div > div.eo2As > div.EtaWk > ul
    try : 
        post_content = driver.find_element_by_css_selector("div.C7I1f.X7jCj > div.C4VMK > span").text    
    except :
        post_content = None
    # 정제 필요
    print("post_content : ", post_content)
    
    # 본문 속 태그
    try :
        tag_list = []
        tag_raw = driver.find_element_by_css_selector(".C7I1f.X7jCj" ).text
        tag = re.findall("#[A-Za-z0-9가-힣]+", tag_raw)
        tag = ''.join(tag).replace("#"," ") # "# 제거"
        
        tag_split = tag.split()
        
        for tag_data in tag_split :
            tag_list.append(tag_data)
        
    except :
        tag_list = []
    print("tag : ", tag)
       
    # 다음 버튼
    next_post_button = driver.find_element_by_css_selector("div.RnEpo._Yhr4 > div.Z2Inc._7c9RR > div > div.l8mY4.feth3 > button")
    next_post_button.click()
    time.sleep(0.5)
    
exit_button = driver.find_element_by_css_selector("div.RnEpo._Yhr4 > div.NOTWr > button > div > svg")
exit_button.click()

date :  2021년 8월 29일
place :  낙산사
post_content :  개강 전 finale
.
.
.
.
.
.
.
.
.
.
혜진 귤 왜 안까 ?
tag :  
date :  2021년 8월 8일
place :  갤러리아 광교, Galleria Gwanggyo
post_content :  곱상한 척✨
.
.
.
.
.
.
.
.
.
.
디저트 폭주 예정❣️
tag :  
date :  2021년 7월 11일
place :  하이서울유스호스텔 Hi Seoul Youth Hostel
post_content :  나도 기본캠📸플래쉬 감성 ?
.
.
.
.
.
.
.
.
포둉포둉한거바🐷
tag :  
date :  2021년 7월 11일
place :  하이서울유스호스텔 Hi Seoul Youth Hostel
post_content :  글램핑
.
.
.
.
.
.
.
.
.
.
.
.
인 척 ….
tag :  
date :  2021년 5월 6일
place :  익선동
post_content :  간만에 🙈
.
.
.
.
.
.
.
.
.
머리끈 너무 거슬리는데
tag :  
date :  2021년 3월 15일
place :  Bar:eve - 바이브
post_content :  이 포즈 이제 컨셉✨
tag :  
date :  2020년 11월 11일
place :  롯데월드 (lotteworld)
post_content :  늙어서 많이 못놀게따🙈
.
.
.
.
.
.
.
.
나는 언제 남자친구랑 ?????
tag :  
date :  2020년 9월 2일
place :  모이핀
post_content :  안 올수가 없는 카페야
.
.
.
.
.
.
.
서울보다 좋아❣️
tag :  
date :  2020년 8월 12일
place :  해운대 - Haeundae
post_content :  바다는 힐링🌙
.
.
.
.
.
.
.
근데 비와서

## 수정 필요 댓글 크롤링